# Analyze ALARO GRIB files

This notebook reads all `.grb` files in the current directory and summarizes their contents (variables, units, etc.) into a table.

In [14]:
import xarray as xr
import pandas as pd
from pathlib import Path
import warnings
import urllib.request
import re

# Suppress warnings for cleaner output (optional)
warnings.filterwarnings('ignore')

In [15]:
def fetch_alaro_params(url):
    print(f"Fetching parameters from {url}...")
    try:
        with urllib.request.urlopen(url) as response:
            content = response.read().decode('utf-8')
        
        params = {}
        # The file seems to be formatted as ID:Abbrev:Description
        # We'll parse line by line.
        
        for line in content.splitlines():
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            
            # Split by colon, limit to 2 splits (3 parts)
            parts = line.split(':', 2)
            
            if len(parts) >= 2:
                try:
                    pid = int(parts[0])
                    abbrev = parts[1].strip()
                    desc = parts[2].strip() if len(parts) > 2 else "N/A"
                    
                    # Store keyed by abbreviation (e.g. '10U', '2T', 'TotPrecip')
                    params[abbrev] = {
                        'id': pid,
                        'description': desc
                    }
                except ValueError:
                    # Skip lines where first part is not an int
                    continue
                    
        print(f"Loaded {len(params)} parameter definitions.")
        return params
    except Exception as e:
        print(f"Error fetching parameters: {e}")
        return {}

alaro_params_url = "https://opendata.meteo.be/resources/forecasts/alaro/params227-228.tab"
alaro_mapping = fetch_alaro_params(alaro_params_url)

Fetching parameters from https://opendata.meteo.be/resources/forecasts/alaro/params227-228.tab...
Loaded 217 parameter definitions.


In [16]:
def analyze_grib_files(directory='.', param_mapping=None):
    path = Path(directory)
    # Ensure re is imported if not already, though it's better at top level
    import re
    
    grib_files = list(path.glob('*.grb'))
    
    results = []
    
    print(f"Found {len(grib_files)} .grb files.")
    
    for file_path in grib_files:
        
        # Parse variable tag from filename (e.g. alaro40l_2026012900_10U.grb -> 10U)
        # Using regex to find the last part separated by underscore before extension
        # pattern: ..._TAG.grb
        file_tag = "N/A"
        match = re.search(r'_([^_]+)$', file_path.stem)
        if match:
            file_tag = match.group(1)
            
        # Lookup expected data from mapping based on filename tag
        expected_desc = "N/A"
        expected_id = "N/A"
        
        if param_mapping and file_tag in param_mapping:
            entry = param_mapping[file_tag]
            expected_desc = entry['description']
            expected_id = entry['id']
            
        try:
            # Open dataset with cfgrib engine
            with xr.open_dataset(file_path, engine='cfgrib', indexpath='') as ds:
                
                # Iterate through data variables in the dataset
                for var_name, da in ds.data_vars.items():
                    
                    # Calculate mean value
                    try:
                        mean_val = float(da.mean().values)
                    except Exception:
                        mean_val = None

                    info = {
                        'file_name': file_path.name,
                        'file_tag': file_tag,
                        'expected_desc': expected_desc,
                        'expected_id': expected_id,
                        'variable_name': var_name,
                        'mean_value': mean_val,
                        'long_name': da.attrs.get('long_name', 'N/A'),
                        'units': da.attrs.get('units', 'N/A'),
                        'shape': da.shape,
                        'stepType': da.attrs.get('GRIB_stepType', 'N/A'),
                        'shortName': da.attrs.get('GRIB_shortName', 'N/A'),
                        'paramId': da.attrs.get('GRIB_paramId', 'N/A')
                    }
                    results.append(info)
                    
        except Exception as e:
            print(f"Error reading {file_path.name}: {e}")
            results.append({
                'file_name': file_path.name,
                'file_tag': file_tag,
                'expected_desc': expected_desc,
                'expected_id': expected_id,
                'variable_name': 'ERROR',
                'long_name': str(e)
            })
            
    # Reorder columns
    df = pd.DataFrame(results)
    if not df.empty and 'expected_desc' in df.columns:
        cols = ['file_name', 'file_tag', 'expected_desc', 'variable_name', 'long_name', 'mean_value', 'units', 'expected_id', 'paramId']
        # Add remaining columns
        cols += [c for c in df.columns if c not in cols]
        df = df[cols]
        
    return df

In [17]:
# Run the analysis with the mapping
df = analyze_grib_files('.', param_mapping=alaro_mapping)

# Display the table
display(df)

Found 34 .grb files.
Error reading alaro40l_2026012900_R.grb: multiple values for unique key, try re-open the file with one of:
    filter_by_keys={'typeOfLevel': 'heightAboveGround'}
    filter_by_keys={'typeOfLevel': 'isobaricInhPa'}


,file_name,file_tag,expected_desc,variable_name,long_name,mean_value,units,expected_id,paramId,shape,stepType,shortName
0,alaro40l_2026012900_10U.grb,10U,10 m u wind component [m s**-1],unknown,unknown,-1.755157e+00,unknown,192,0.0,"(61, 177, 177)",instant,unknown
1,alaro40l_2026012900_10V.grb,10V,10 m v wind component [m s**-1],unknown,unknown,2.711944e+00,unknown,193,0.0,"(61, 177, 177)",instant,unknown
2,alaro40l_2026012900_2D.grb,2D,2 m dewpoint temperature [K or C],unknown,unknown,2.754382e+02,unknown,195,0.0,"(61, 177, 177)",instant,unknown
3,alaro40l_2026012900_2T.grb,2T,2 m temperature [K or C],unknown,unknown,2.763112e+02,unknown,194,0.0,"(61, 177, 177)",instant,unknown
4,alaro40l_2026012900_CR.grb,CR,Convective rain [m],strdc,Surface long-wave (thermal) radiation downward...,4.499003e-04,J m**-2,130,228130.0,"(61, 177, 177)",instant,strdc
5,alaro40l_2026012900_CS.grb,CS,Convective snow [m],u10n,10 metre u-component of neutral wind,6.326314e-05,m s**-1,131,228131.0,"(61, 177, 177)",instant,u10n
6,alaro40l_2026012900_fzht.grb,fzht,freezing level [m],unknown,unknown,8.484856e+02,unknown,234,0.0,"(61, 177, 177)",instant,unknown
7,alaro40l_2026012900_IFCCC.grb,IFCCC,Inst flx Conv Cld Cover [0-1],unknown,unknown,1.610348e-01,unknown,213,0.0,"(61, 177, 177)",instant,unknown
8,alaro40l_2026012900_IFHCC.grb,IFHCC,Inst flx High Cld Cover [0-1],fzra,Accumulated freezing rain,3.494167e-01,m,216,228216.0,"(61, 177, 177)",instant,fzra
9,alaro40l_2026012900_IFLCC.grb,IFLCC,Inst flx Low Cld Cover [0-1],unknown,unknown,8.192638e-01,unknown,214,0.0,"(61, 177, 177)",instant,unknown


In [18]:
# Export to CSV if needed
# df.to_csv('alaro_variables_summary.csv', index=False)